In [3]:
# Import libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the saved model
model_path = 'specify the path for the model'
model_state_dict = torch.load(model_path)

# Initialize the model architecture
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming binary classification
model.load_state_dict(model_state_dict)
model.eval()

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
### Define the function for humor prediction ###

def predict_humor(text, tokenizer, model):

    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class
    predicted_class = torch.argmax(outputs.logits, dim=1).item()

    # Assuming class 0 represents non-humorous and class 1 represents humorous
    if predicted_class == 0:
        return "The text is predicted as non-humorous."
    elif predicted_class == 1:
        return "The text is predicted as humorous."
    else:
        return "Unknown prediction class."

In [13]:
# first try (from "Friends"):

text1 = "Okay, everybody, this is Rachel, another Lincoln High survivor." #@param {type:"string"}
result = predict_humor(text1, tokenizer, model)
print(result)

The text is predicted as humorous.


In [14]:
#second try:

text2 = "What a wonderful weather"#@param {type:"string"}
result = predict_humor(text2, tokenizer, model)
print(result)

The text is predicted as non-humorous.


In [15]:
### Let's compare two texts ###

def compare_humor(text1, text2, tokenizer, model):

    # Tokenize the texts
    inputs1 = tokenizer(text1, return_tensors='pt', truncation=True, padding=True)
    inputs2 = tokenizer(text2, return_tensors='pt', truncation=True, padding=True)

    # Make predictions for both texts
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # Get the predicted class probabilities for both texts
    predicted_class1 = torch.softmax(outputs1.logits, dim=1)[0][1].item()
    predicted_class2 = torch.softmax(outputs2.logits, dim=1)[0][1].item()

    # Compare the probabilities to determine the prediction
    if predicted_class2 > predicted_class1:
        return "The first text is predicted as humorous with higher probability."
    elif predicted_class1 > predicted_class2:
        return "The second text is predicted as humorous with higher probability."
    else:
        return "Both texts have equal probabilities of being humorous."

In [16]:
# Example usage (from "The Catcher in the Rye")

text1 = "If you really want to hear about it, the first thing you'll probably want to know is where I was born" #@param {type:"string"}
text2 = "and all that David Copperfield kind of crap" #@param {type:"string"}
result = compare_humor(text1, text2, tokenizer, model)
print(result)

The second text is predicted as humorous with higher probability.
